In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import json as js
import csv

In [2]:
df = pd.read_csv('data/resume.csv')

In [3]:
df.shape

(2369, 20)

In [4]:
df.duplicated().shape

(2369,)

In [5]:
df.head(1)

,Unnamed: 0,age,auto,business_trips,categories,citizenship,city,education,employment,experience,experience_length,family,gender,languages,metro_or_moving,name,photo_url,salary,skills,url
0,0,20,NaN,не готов к командировкам,Архитектура,Россия,Ковров,"[{'kind': 'Неполное высшее образование', 'form...",неполный рабочий день,"[{'period': 'сентябрь 2017 – март 2018', 'leng...",7 месяцев,NaN,f,"['Немецкий язык, Базовый']",NaN,"3D-моделлер, IT стажёр, консультант",NaN,30 000₽,NaN,https://www.superjob.ru/resume/3d-modeller-436...


In [6]:
df.columns

Index(['Unnamed: 0', 'age', 'auto', 'business_trips', 'categories',
       'citizenship', 'city', 'education', 'employment', 'experience',
       'experience_length', 'family', 'gender', 'languages', 'metro_or_moving',
       'name', 'photo_url', 'salary', 'skills', 'url'],
      dtype='object')

In [7]:
df = df.drop('Unnamed: 0', axis=1)

In [8]:
df.auto.value_counts(dropna=False)

['B — легковые авто']                                                                                 1171
NaN                                                                                                    808
['B — легковые авто', 'C — грузовые авто']                                                             220
['A — мотоциклы', 'B — легковые авто']                                                                  74
['A — мотоциклы', 'B — легковые авто', 'C — грузовые авто']                                             52
['C — грузовые авто']                                                                                   11
['B — легковые авто', 'C — грузовые авто', 'D — автобусы']                                              11
['B — легковые авто', 'C — грузовые авто', 'D — автобусы', 'E — авто с прицепом']                        7
['B — легковые авто', 'D — автобусы']                                                                    5
['A — мотоциклы', 'B — легковые авто'

In [9]:
df.auto=df.auto.fillna("['Нет прав']")

## Замена - не выкидываем

In [10]:
df.business_trips.value_counts(dropna=False)

NaN                         1373
не готов к командировкам     968
на дому                       28
Name: business_trips, dtype: int64

In [11]:
df.url.iloc[1]

'https://www.superjob.ru/resume/administrator-26936498.html'

## Замена, ничего не понятно, почему либо не готов, либо на дому, где вариант готов?

In [12]:
#df = df.drop('business_trips', axis=1)

In [13]:
df.categories.value_counts(dropna=False)

Системное администрирование                      552
NaN                                              467
Техническая поддержка                            358
IT, Интернет, связь, телеком                     238
Инжиниринг                                       184
Администрирование баз данных                      80
Административная работа, секретариат, АХО         64
Сетевые технологии                                49
Разработка, программирование                      41
Передача данных и доступ в интернет               38
Управление проектами                              34
Аналитика                                         31
Интернет, создание и поддержка сайтов             23
Другое                                            18
Электронный документооборот                       15
Внедрение и сопровождение ПО                      12
Делопроизводство, ввод данных, систематизация     11
Компьютеры, оргтехника, ПО                         9
Web-верстка                                   

In [14]:
df.categories=df.categories.fillna('Не указано')

Выкинуть незаполненные (20 % данных) или весь признак? Неясно
## Выкинуть всегда успеем, преобразовал в отдельную категорию.

Так как категории заполнялись сайтом автоматически, эту информацию можно извлечь из других полей (name), удаляем.
## Хотя может ты и прав, короче потом подумаем.

In [15]:
df.citizenship.value_counts(dropna=False)

Россия               1615
NaN                   727
Украина                 8
Казахстан               7
Азербайджан             3
Кыргызстан              3
Беларусь                2
Таджикистан             1
Молдова                 1
Дальнее зарубежье       1
Литва                   1
Name: citizenship, dtype: int64

Считаем, что гражданство не влияет на уровень зп.

In [16]:
#df = df[(df.citizenship == 'Россия')  | df.citizenship.isna()]
df = df.drop('citizenship', axis=1)

In [17]:
df.city.value_counts(dropna=False)

Москва                               720
Санкт-Петербург                      293
Хабаровск                             64
Уфа                                   55
Иркутск                               51
Ростов-на-Дону                        48
Краснодар                             33
Самара                                32
Саратов                               27
Казань                                27
Рязань                                26
Нижний Новгород                       24
Пенза                                 23
Ульяновск                             23
Волгоград                             22
Воронеж                               20
Оренбург                              19
Сыктывкар                             17
Тверь                                 17
Смоленск                              16
Владимир                              16
Челябинск                             16
Екатеринбург                          16
Тюмень                                15
Астрахань       

Обучать модель для регионов отдельно не хочется, будет слишком мало объектов. Предложение: выделить категории в зависимости от уровня зарплат по региону. Для начала 3: Мск, Спб, остальные.

In [18]:
ctgr = pd.get_dummies(df.city)
city_Msk = ctgr['Москва']
city_Spb = ctgr['Санкт-Петербург']
ctgr = ctgr.drop(['Москва', 'Санкт-Петербург'], axis=1)
city_oth = np.sum(ctgr.values, axis=1)
df['city_Msk'] = city_Msk
df['city_Spb'] = city_Spb
df['city_oth'] = city_oth
df = df.drop(['city'], axis=1)
df.shape

(2369, 20)

In [19]:
df.employment.value_counts(dropna=False)

полная занятость           1180
NaN                        1107
неполный рабочий день        33
сменный график работы        25
частичная занятость          13
работа вахтовым методом      11
Name: employment, dtype: int64

Много NaN, выкидываем.
## Очень важная же инфа

In [20]:
df.employment=df.employment.fillna("Не указано")

In [21]:
df.family.value_counts(dropna=False)

cостоит в браке, есть дети       804
NaN                              687
не состоит в браке, детей нет    528
cостоит в браке, детей нет       168
не состоит в браке, есть дети    111
cостоит в браке                   37
не состоит в браке                29
есть дети                          3
детей нет                          2
Name: family, dtype: int64

In [22]:
df = df.drop('family', axis=1)

In [23]:
gender = pd.get_dummies(df.gender)
gender.f
df['gender'] = gender.f

Как вариант можно считать, что если иностранные языки не указаны, то человек ими не владеет. Логика в этом есть!!!

In [24]:
df.languages.value_counts(dropna=False)

['Английский язык, Технический']                                                                                                                     832
NaN                                                                                                                                                  547
['Английский язык, Базовый']                                                                                                                         463
['Английский язык, Разговорный']                                                                                                                     164
['Английский язык, Технический', 'Немецкий язык, Базовый']                                                                                            47
['Английский язык, Базовый', 'Немецкий язык, Базовый']                                                                                                42
['Английский язык, Cвободное владение']                                           

In [25]:
#df = df.drop('languages', axis=1)
df.languages=df.languages.fillna('')

In [61]:
type(df.languages.iloc[1])

str

In [67]:
Lang = set()
for x in df.languages.values:
    if type(x)==float:
        Lang.add('С')
    print (x)
    for y in eval(x):
        Lang.add(y)
Lang

SyntaxError: unexpected EOF while parsing (<string>, line 0)

In [26]:
df.metro_or_moving.value_counts(dropna=False)

NaN                              1357
готов к переезду                  540
Пражская                           12
Улица Дыбенко                       9
Комендантский проспект              8
Ладожская                           6
Коломенская                         6
Новогиреево                         6
Ботанический садВДНХ                6
Медведково                          6
Первомайская                        6
Выхино                              6
Проспект Ветеранов                  6
Академическая                       5
Щелковская                          5
Новокосино                          5
Планерная                           5
Проспект Большевиков                5
Алтуфьево                           4
Московская                          4
Бульвар Рокоссовского               4
Войковская                          4
Перово                              4
Строгино                            4
Бибирево                            4
Митино                              4
Пионерская и

## Не сделано, но может быть лучше рабить на готов-не готов к переезду?

In [27]:
#df = df.drop('metro_or_moving', axis=1)

In [28]:
df.name.value_counts(dropna=False)

Системный администратор, IT-специалист                                                                                  416
IT-специалист                                                                                                           207
IT специалист                                                                                                            76
ИТ-специалист                                                                                                            54
Руководитель IT-отдела                                                                                                   37
Инженер IT                                                                                                               29
Руководитель отдела ИТ                                                                                                   27
Руководитель IT отдела                                                                                                   24
It-специ

In [29]:
df.name.unique().shape

(1115,)

Нужно обрабатывать как текст. Очень много синонимов!

In [30]:
df = df.drop('photo_url', axis=1)

In [31]:
df.skills.value_counts(dropna=False).shape

(1965,)

Часть - дубликаты. Но полных дубликатов не было. Видимо, люди хотят на разные должности по одинаковым резюме. Можно оставить как есть (логике задачи это не противоречит)

In [32]:
df.groupby(by='skills')

In [33]:
df[df.skills.isna()].shape

(330, 18)

In [34]:
df = df.dropna(subset=['skills'])

## Я плох в панде, что здесь происходит?

Видимо, придется выкинуть незаполненные, важная инфа :(

In [35]:
df = df.drop('url', axis = 1)

In [36]:
df.shape

(2039, 17)

In [37]:
df.education.value_counts(dropna=False)

NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    43
[{'kind': 'Среднее образование'}]                                                                                                                                                             

## Тяжело
нашлись уникумумы с 4 образованиями

In [38]:
A=set()

In [39]:
A=set()
for x in df.education.values:
    if type(x)==float:
        A.add("Пусто")
    else:
        for ed in eval(x):
            if ed['kind']=='Учащийся школы':
                print(x)
            A.add(ed['kind'])

[{'kind': 'Высшее образование', 'form': 'Очно-заочная', 'period': '2008', 'university': 'Международная промышленная академия', 'faculty': 'Промышленная безопасность и охрана труда на предприятиях агропромышленного комплекса', 'specialty': 'Инженер по промышленной безопасности и охране труда'}, {'kind': 'Среднее специальное образование', 'form': 'Дневная/Очная', 'period': '1989', 'university': 'Вооружённые силы СССР', 'specialty': 'Сержант, командир отделения'}, {'kind': 'Среднее специальное образование', 'form': 'Дневная/Очная', 'period': '1987', 'university': 'Московский Политехникум Министерства Заготовок РСФСР', 'specialty': 'Техник-механик'}, {'kind': 'Учащийся школы'}]
[{'kind': 'Бакалавр', 'form': 'Заочная', 'period': '2020', 'university': 'Международный Университет в Москве', 'faculty': 'Предпринимательство в Культуре', 'specialty': 'Связи с общественностью и реклама'}, {'kind': 'Неполное высшее образование', 'form': 'Заочная', 'period': '2007', 'university': 'Международный Унив

In [40]:
A

{'Бакалавр',
 'Высшее образование',
 'Кандидат наук',
 'Магистр',
 'Неполное высшее образование',
 'Пусто',
 'Среднее образование',
 'Среднее специальное образование',
 'Учащийся школы'}

In [58]:
df.education.value_counts(dropna=False)

[{'kind': 'Среднее образование'}]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      21
[{'kind': 'Высшее образование', 'form': 'Дневная/Очная', 'period': '1985', 'university': 'Харьковское высшее военное авиационное училище радиоэлектроники', 'faculty': 'Командный', 'specialty

Просто интересно.

def returner_because_i_am_not_clever(x):
    if type(x)==float:
        return "пусто"
    if len(eval(x))==4:
        #print (x)
        print ("^^^^^^^^^^
    
    return eval()
set([returner_because_i_am_not_clever(x) for x in df.education.values])

In [41]:
#df = df[df.education.isna() == False]
df = df.dropna(subset=['education'])

Выкидываем незаполненные. Че дальше - непонятно

In [42]:
df.experience_length.value_counts(dropna=False)

NaN                     78
7 лет и 3 месяца        17
14 лет и 4 месяца       17
10 лет и 6 месяцев      14
12 лет и 8 месяцев      13
9 лет и 7 месяцев       13
3 года и 6 месяцев      13
11 лет и 4 месяца       13
11 лет и 11 месяцев     13
10 лет и 2 месяца       13
13 лет и 11 месяцев     12
7 лет и 2 месяца        12
5 лет и 10 месяцев      11
7 лет и 9 месяцев       11
12 лет и 5 месяцев      11
13 лет и 4 месяца       11
12 лет и 11 месяцев     11
6 лет и 6 месяцев       11
9 лет и 5 месяцев       11
15 лет и 5 месяцев      11
7 лет и 8 месяцев       11
16 лет и 5 месяцев      11
15 лет и 2 месяца       11
9 лет и 6 месяцев       11
13 лет и 2 месяца       11
10 лет и 3 месяца       11
8 лет и 6 месяцев       11
10 лет и 4 месяца       10
11 лет и 10 месяцев     10
9 лет и 8 месяцев       10
                        ..
35 лет и 5 месяцев       1
34 года и 4 месяца       1
25 лет и 3 месяца        1
33 года и 5 месяцев      1
20 лет и 1 месяц         1
24 года и 5 месяцев      1
2

Наны, видимо, - нет опыта работы

In [43]:
df.experience_length=df.experience_length.fillna("0 лет и 0 месяцев")

In [44]:
#df = df.dropna(subset=['experience_length'])

In [45]:
df.shape

(1996, 17)

In [46]:
df.index = np.arange(df.shape[0])

In [47]:
df.columns

Index(['age', 'auto', 'business_trips', 'categories', 'education',
       'employment', 'experience', 'experience_length', 'gender', 'languages',
       'metro_or_moving', 'name', 'salary', 'skills', 'city_Msk', 'city_Spb',
       'city_oth'],
      dtype='object')

### Промежуточный итог: ###

Остались пропуски в 'languages', нужно решить, что с ними делать. Идея в том, что NaN можно интерпретировать как незнание языков. 

В остальных фичах пропусков нет. 

Город и пол one-hot.

Что с образованием - неясно, очень много категорий.

Для 'experience' и 'skills' нужно посмотреть распределения N-грамм и выбрать хорошие, информативные фичи по вхождениям ключевых слов.

Так же необходимо все числовые признаки нормализовать.

In [48]:
def period_to_month(string):
    s = string.split()
    if len(s) == 2:
        if (s[1] == 'год') or (s[1] == 'года') or (s[1] == 'лет'):
            return int(s[0]) * 12
        return int(s[0])
    return int(s[0]) * 12 + int(s[3])

In [49]:
pers = df.experience_length.copy()

In [50]:
pers is df.experience_length

False

In [51]:
for i in pers.index:
    pers.values[i] = period_to_month(pers.values[i])

In [52]:
df.experience_length = pers
df.experience_length

0       216
1       154
2        51
3        14
4        28
5        55
6        51
7        59
8       224
9         3
10       78
11      172
12      361
13      411
14      111
15      250
16        0
17      183
18      284
19      216
20      280
21      381
22      186
23      126
24      128
25       42
26      268
27      167
28      334
29      330
       ... 
1966    136
1967    121
1968    283
1969    283
1970    207
1971    244
1972    167
1973    121
1974    179
1975    165
1976    389
1977    334
1978    245
1979     97
1980    194
1981    211
1982    154
1983    268
1984     75
1985    298
1986    156
1987    194
1988    339
1989    156
1990    146
1991     66
1992    180
1993    347
1994     34
1995     72
Name: experience_length, Length: 1996, dtype: object

Перевели стаж работы в месяцы.

In [53]:
for x in df.salary.values:
    if (x[-1]!='₽'):
        print (x)

ЗП только в рублях, что очень странно

In [54]:
df.salary=df.salary.apply(lambda x: int(x[:-1].replace(" ", '')))

In [55]:
df.head(3)

,age,auto,business_trips,categories,education,employment,experience,experience_length,gender,languages,metro_or_moving,name,salary,skills,city_Msk,city_Spb,city_oth
0,39,['B — легковые авто'],NaN,Техническая поддержка,"[{'kind': 'Высшее образование', 'form': 'Дневн...",полная занятость,"[{'period': 'октябрь 2017 – работает сейчас', ...",216,0,,NaN,"Администратор, управляющий, IT специалист",50000,[{'professional': '• 1С: Предприятие • 1С: Роз...,1,0,0
1,47,['Нет прав'],NaN,Передача данных и доступ в интернет,"[{'kind': 'Высшее образование', 'form': 'Вечер...",полная занятость,"[{'period': 'февраль 2008 – работает сейчас', ...",154,0,"['Английский язык, Технический']",Берёзовая роща,Администратор / Инженер технической поддержки IT,30000,[{'professional': 'Английский язык уровень. Pr...,0,0,1
2,23,['Нет прав'],NaN,Техническая поддержка,"[{'kind': 'Высшее образование', 'form': 'Дневн...",сменный график работы,"[{'period': 'июль 2015 – работает сейчас', 'le...",51,1,,NaN,"Администратор, IT-специалист, бизнес-аналитик,...",25000,"[{'professional': 'Веб-дизайн, photoshop, MS O...",0,0,1


In [56]:
df.salary.value_counts

<bound method IndexOpsMixin.value_counts of 0        50000
1        30000
2        25000
3        45000
4        20000
5        45000
6        30000
7        20000
8        30000
9        35000
10       60000
11       30000
12       75000
13       75000
14       20000
15      150000
16       25000
17      150000
18       70000
19      280000
20      240000
21       90000
22       80000
23       20000
24       50000
25       50000
26      100000
27      150000
28      110000
29      150000
         ...  
1966     40000
1967    175000
1968     80000
1969     80000
1970    150000
1971    200000
1972     50000
1973    120000
1974     70000
1975    130000
1976     50000
1977    120000
1978     55000
1979     50000
1980    100000
1981    150000
1982    120000
1983     50000
1984     25000
1985     60000
1986    100000
1987     50000
1988     50000
1989     60000
1990    110000
1991     30000
1992    150000
1993    150000
1994     45000
1995     80000
Name: salary, Length: 1996, dtype: int64>